# Munich Airbnb Data Analysis

## 1. Introduction 

**Dataset**: The used datasets were created on July 17th, 2021 and contain detailed listings data, review data and calendar data of current Airbnb listings in Munich, Germany. This data was created by Murray Cox and his Inside Airbnb project which can be found here: http://insideairbnb.com/get-the-data.html

**Methodology**: For the analysis I will use the CRISP-DM Methodology. CRIPS-DM stands for "cross-industry process for data mining". The process consists of six steps:

1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment

## 2. Business Understanding

**Airbnb** is a community-driven platform that promotes amazing travel that is local, genuine, and one-of-a-kind. Airbnb has over 5 million listings and is present in 191 countries and over 81,000 cities. Through a concept intended to promote healthy travel, the Airbnb community has hosted over half a billion visitors to date.

**Research questions**:
1. What are the TOP 3 most expensive city districts for a stay in Munich in general?
2. What are the TOP 3 factors influence the price for a stay?

In [ ]:
# Import required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn import metrics
%matplotlib inline

## 3. Data Understanding

### 3.1 Load the datasets

The data set contains the following files:
* listings.csv: descriptions and review score
* calendar.csv: listing id, price and availability for the upcoming year
* reviews.csv: unique id for each reviewer and detailed comments

In [ ]:
df_listings = pd.read_csv("data/listings.csv");
df_calendar = pd.read_csv("data/calendar.csv");
df_reviews = pd.read_csv("data/reviews.csv")

### 3.2 A glimpse of the data

In this section, we examine the overall features of each dataset

#### The "Calendar" dataset

In [ ]:
# Take a look at a concise summary of the DataFrame 'calendar'
df_calendar.info()

In [ ]:
# Show the first rows of the dataframe
df_calendar.head()

In [ ]:
# List all features in this data set and show the number of missing values
obj = df_calendar.isnull().sum()
for key,value in obj.iteritems():
    percent = round((value * 100 / df_calendar['listing_id'].index.size),3)
    print(key,", ",value, "(", percent ,"%)")

In [ ]:
# Show the shape of the dataframe
df_calendar.shape

The data set consists of 7 features and a total rows of 1.860.770

Only the features 'price' and 'adjusted price' contain missing data (both 180).

The following data cleaning is suggested for further analysis:
* Drop feature 'adjusted_price'.
* Remove the rows in 'price' that have missing data (the number of impacted rows is minimal, 0.01 percent ).
* The feature 'price' must be transformed to a numerical data type.
* The 'date' feature must be changed to datetime format.
* The attribute 'available' must be changed to a bool data type.

#### The "Listings" dataset

In [ ]:
# Take a closer look at a concise summary of the DataFrame 'listings'
df_listings.info()

In [ ]:
# Show the first rows of the dataframe
df_listings.head()

In [ ]:
# List all features in this data set and show the number of missing values
obj = df_listings.isnull().sum()
for key,value in obj.iteritems():
    percent = round((value * 100 / df_listings['id'].index.size),3)
    print(key,", ",value, "(", percent ,"%)")

In [ ]:
# Show the shape of the dataframe
df_listings.shape

In [ ]:
# Count distinct observations per feature
df_listings.nunique()

The dataset consists of 74 features and a total rows of 5098.

The overall quality is not satisfactory for all aspects. It must be decided how to handle the remaining missing data.

Filling does not seem to be very promising in all instances, therefore the characteristics should be removed from a threshold.

Some features have only constant values and don't help us any further.

The following data cleaning is suggested for further analysis:

* Drop features with constant values
* Drop features with more than 50% missing data
* Fill missing numerical data with mean value
* Convert features to useable data type (e.g. price)
* Drop features that do not provide us with any useful information (for our specific questions)

#### The "Reviews" dataset

In [ ]:
# Take a look at a concise summary of the DataFrame 'reviews'
df_reviews.info()

In [ ]:
# Show the first rows of the dataframe
df_reviews.head()

In [ ]:
# List all features in this data set and show the number of missing values
obj = df_reviews.isnull().sum()
for key,value in obj.iteritems():
    percent = round((value * 100 / df_reviews['id'].index.size),3)
    print(key,", ",value, "(", percent ,"%)")

In [ ]:
# Show the shape of the dataframe
df_reviews.shape

The dataset consists of 6 features and a total of 106.971 rows.

The overall quality is good, only the feature 'comments' has missing data (72).

At the first sight the data set cannot be used to answer the questions. Nevertheless, the number of reviews shows an interesting pattern which should also be examined.

The feature 'id' may be connected to the other datasets 'calendar' or 'listings'.

Missing data in 'comments' can be dropped (number of affected rows is low ~0.04%) and the feature 'date' should be converted into 'DateTime' data type

## 4. Data Preparation

### 4.1 Feature Selection

In order to address our business questions, we just need a few precise and important characteristics.

**Location**:
* neighbourhood_cleansed
* latitude
* longitude

**Room**:
* room_type
* minimum_nights
* maximum_nights
* amenities

**Price**:
* price

**Score**:
* review_scores_rating
* review_scores_accuracy
* review_scores_cleanliness
* review_scores_checkin
* review_scores_communication
* review_scores_location
* review_scores_value
* reviews_per_month

### 4.2 Question 1: "What are the TOP 3 most expensive city districts for a stay in Munich in general?"

First, we clean the "listings" dataset

In [ ]:
# Copy the data to a new DataFrame
df_listings_clean = df_listings.copy(deep=True)

In [ ]:
# Clean up the data set "listings" according to our previous analysis

# Drop unnecessary features
features_to_drop = ['listing_url', 'picture_url','host_url', 'host_thumbnail_url', 'host_picture_url',
                    'name', 'neighborhood_overview', 'description',
                    'host_name', 'host_location', 'host_neighbourhood', 'last_scraped',
                    'calendar_last_scraped', 'first_review', 'last_review', 'host_since', 'calendar_updated',
                    'host_total_listings_count']
df_listings_clean.drop(features_to_drop, axis=1, inplace=True)

In [ ]:
# Remove constant features by finding unique values per feature 
df_listings_clean = df_listings_clean[df_listings_clean.nunique().where(df_listings_clean.nunique()!=1).dropna().keys()]

# Drop features with 50% or more missing values
more_than_50 = list(df_listings_clean.columns[df_listings_clean.isnull().mean() > 0.5])
df_listings_clean.drop(more_than_50, axis=1, inplace=True)

# Clean up the format values
df_listings_clean['price'] = df_listings_clean['price'].replace('[\$,]', '', regex=True).astype(float)

# Convert rates type from string to float and remove the % sign
df_listings_clean['host_response_rate'] = df_listings_clean['host_response_rate'].astype(str).str.replace('%', '').astype(float)
df_listings_clean['host_response_rate'] = df_listings_clean['host_response_rate'] * 0.01
df_listings_clean['host_acceptance_rate'] = df_listings_clean['host_acceptance_rate'].astype(str).str.replace('%', '').astype(float)
df_listings_clean['host_acceptance_rate'] = df_listings_clean['host_acceptance_rate'] * 0.01

# Covert boolean data from string data type to boolean
boolean_features = ['instant_bookable', 'host_is_superhost', 'host_has_profile_pic', 
                'host_identity_verified']
df_listings_clean[boolean_features] = df_listings_clean[boolean_features].replace({'t': True, 'f': False})

# Fill numerical missing data with mean value
numerical_feature = df_listings_clean.select_dtypes(np.number)
numerical_columns = numerical_feature.columns

imp_mean = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imp_mean = imp_mean.fit(numerical_feature)

df_listings_clean[numerical_columns] = imp_mean.transform(df_listings_clean[numerical_columns])
     
# Remove all remaining missing values  
df_listings_clean.dropna(inplace=True)

In [ ]:
# Show mean price for each neighbourhood_cleansed
df_listings_clean.groupby(["neighbourhood_cleansed"])["price"].describe().sort_values("mean", ascending=False)

In [ ]:
# Create new feature 'mean' with the mean price per neighbourhood
df_listings_clean['mean'] = df_listings_clean.groupby('neighbourhood_cleansed')['price'].transform(lambda r : r.mean())

In [ ]:
# Create plot for mean price per neighbourhood 
df_listings_plot = df_listings_clean
df_listings_plot = df_listings_plot.groupby('neighbourhood_cleansed')[['price']].mean()
df_listings_plot = df_listings_plot.reset_index()
df_listings_plot = df_listings_plot.sort_values(by='price',ascending=False)
df_listings_plot.plot.bar(x='neighbourhood_cleansed', y='price', color='blue', rot=90, figsize = (20,10)).set_title('Mean Price per city district (Neighbourhood)');

In [ ]:
# Create a map using the latitude and longitude data
fig = px.scatter_mapbox(df_listings_clean, color="mean", lat='latitude', lon='longitude',
                        center=dict(lat=48.137154, lon=11.576124), zoom=10,
                        mapbox_style="open-street-map",width=1000, height=800);
fig.show()

**Answer for Question 1**

The closer the city to the center of Munich, the higher the price
The top 3 most expensive districts to stay in:
* Altstadt-Lehel
* Ludwigsvorstadt-Isarvorstadt
* Tudering-Riem

# 5. Modeling

In [ ]:
# Copy the data to a new DataFrame for encoding 
df_listings_encoded = df_listings_clean.copy(deep=True)

In [ ]:
df_listings_encoded.head()

In [ ]:
# Encode features for use in machine learing model

# Encode feature 'amenities' and concat the data
df_listings_encoded.amenities = df_listings_encoded.amenities.str.replace('[{""}]', "")
df_amenities = df_listings_encoded.amenities.str.get_dummies(sep = ",")
df_listings_encoded = pd.concat([df_listings_encoded, df_amenities], axis=1) 

# Encode feature 'host_verification' and concat the data
df_listings_encoded.host_verifications = df_listings_encoded.host_verifications.str.replace("['']", "")
df_verification = df_listings_encoded.host_verifications.str.get_dummies(sep = ",")
df_listings_encoded = pd.concat([df_listings_encoded, df_verification], axis=1)
    
# Encode feature 'host_response_time'
dict_response_time = {'within an hour': 1, 'within a few hours': 2, 'within a day': 3, 'a few days or more': 4}
df_listings_encoded['host_response_time'] = df_listings_encoded['host_response_time'].map(dict_response_time)

# Encode the remaining categorical feature 
for categorical_feature in ['neighbourhood_cleansed', 'property_type', 'room_type',  'neighbourhood']:
    df_listings_encoded = pd.concat([df_listings_encoded, 
                                     pd.get_dummies(df_listings_encoded[categorical_feature])],axis=1)

# Drop features
df_listings_encoded.drop(['amenities', 'neighbourhood_cleansed', 'property_type', 'room_type', 
                          'host_verifications', 'neighbourhood',
                          'id', 'host_id', 'mean', 'latitude', 'longitude'],
                         axis=1, inplace=True)

In [ ]:
# Last check if there are any missing values in the data set
sum(df_listings_encoded.isnull().sum())

In [ ]:
# Shuffle the data to ensure a good distribution
df_listings_encoded = shuffle(df_listings_encoded)

X = df_listings_encoded.drop(['price'], axis=1)
y = df_listings_encoded['price']

# Split the data into random train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# Control shape after train-test-split
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)